In [1]:
import os.path
from gensim import downloader
import numpy as np
import re

In [10]:
WORD_2_VEC_PATH = 'word2vec-google-news-300'
word2vec_google = downloader.load(WORD_2_VEC_PATH)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [16]:
word2vec_google.save_word2vec_format("word2vec_google.model")

OSError: [Errno 28] No space left on device

In [158]:
word2vec_google.init_sims(replace=True)

C:\Users\shirel\AppData\Local\Temp\ipykernel_169868\2835204304.py:1: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  word2vec_google.init_sims(replace=True)


In [159]:
print(word2vec_google)

KeyedVectors<vector_size=300, 3000000 keys>


In [124]:
def preprocess(data_path, tagged):
    # preprocess data
    train_path = os.path.join('data', data_path) #'train.tagged' pour simon sur collab
    list_of_sentences_with_tags = []
    list_of_words = []
    sentence_index = 0
    list_of_sentences =[]
    prob_line = 0
    with open(train_path, encoding='utf-8') as f:
        for line in f:
            prob_line+=1
            if line == "\t\n" or line == "\n":
                sentence_index += 1
                continue
            if line[-1:] == "\n":
                line = line[:-1]
            try :
                word, tag = line.split('\t')
            except:
                break
            word = word.strip().lower()
            tag = 0 if tag=='O' else 1
            #word = re.sub(r'\W+', '', word)
            list_of_words.append((word, tag))
            if len(list_of_sentences_with_tags) <= sentence_index:
                list_of_sentences_with_tags.append([(word, tag)])
                list_of_sentences.append([word])

            else:
                list_of_sentences_with_tags[sentence_index].append((word, tag))
                list_of_sentences[sentence_index].append(word)
            # if word.startswith('@') and tag !='O':
            #     print(word, tag)
    return list_of_sentences, list_of_sentences_with_tags, list_of_words




In [125]:
list_of_sentences, list_of_sentences_with_tags, list_of_words = preprocess("train.tagged", True)
print(list_of_sentences)

[['@paulwalk', 'it', "'s", 'the', 'view', 'from', 'where', 'i', "'m", 'living', 'for', 'two', 'weeks', '.', 'empire', 'state', 'building', '=', 'esb', '.', 'pretty', 'bad', 'storm', 'here', 'last', 'evening', '.'], ['from', 'green', 'newsfeed', ':', 'ahfa', 'extends', 'deadline', 'for', 'sage', 'award', 'to', 'nov', '.', '5', 'http://tinyurl.com/24agj38'], ['pxleyes', 'top', '50', 'photography', 'contest', 'pictures', 'of', 'august', '2010', '...', 'http://bit.ly/bgcyz0', '#photography'], ['today', 'is', 'my', 'last', 'day', 'at', 'the', 'office', '.'], ['4dbling', "'s", 'place', 'til', 'monday', ',', 'party', 'party', 'party', '.', '&lt;', '3'], ['watching', 'the', 'vma', 'pre-show', 'again', 'lol', 'it', 'was', "n't", 'even', 'a', 'good', 'show', 'the', 'first', 'time', '...', 'so', 'bored', '!'], ['27', 'followers', '!', '30', 'followers', 'is', 'my', 'goal', 'for', 'today', '!'], ['this', 'is', 'the', '2nd', 'hospital', 'ive', 'been', 'in', 'today', ',', 'but', 'ive', 'just', 'seen

In [152]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=list_of_sentences, vector_size=300, window=2, min_count=1, workers=4, epochs=100, seed=42)
model.init_sims(replace=True)
model.save("word2vec.model")

C:\Users\shirel\AppData\Local\Temp\ipykernel_169868\2917302621.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [161]:
# represent data with glove
representation = []
labels = []
google = 0
model_nous = 0
for word, tag in list_of_words:
    if word not in word2vec_google.key_to_index:
        if tag:
            print(f"{word} // {tag} not an existing word in the model")
        vec = model.wv[word]
    else:
        vec = word2vec_google[word]
        google +=1
    labels.append(tag)
    representation.append(vec)

esb // 1 not an existing word in the model
ahfa // 1 not an existing word in the model
pxleyes // 1 not an existing word in the model
4dbling // 1 not an existing word in the model
vma // 1 not an existing word in the model
metropcs // 1 not an existing word in the model
's // 1 not an existing word in the model
tanya // 1 not an existing word in the model
s.f. // 1 not an existing word in the model
#battlestargalactica // 1 not an existing word in the model
#starwars // 1 not an existing word in the model
#theclonewars // 1 not an existing word in the model
alderwood // 1 not an existing word in the model
icarly // 1 not an existing word in the model
wintor // 1 not an existing word in the model
of // 1 not an existing word in the model
g24 // 1 not an existing word in the model
aintree // 1 not an existing word in the model
wingo // 1 not an existing word in the model
gainesville // 1 not an existing word in the model
#padres // 1 not an existing word in the model
@ravenalexis // 1 n

In [162]:
representation_array = np.asarray(representation)
print(representation_array.shape)

(62730, 300)


In [163]:
print(representation[0])

[-0.06499673 -0.00358599 -0.08962213 -0.08995916 -0.01960194 -0.01221613
  0.07265271  0.04988514 -0.0456614   0.01672189  0.00377306  0.06766422
  0.01720975  0.01100329  0.07286973 -0.00452309 -0.05330775 -0.05187264
  0.002012   -0.02114924  0.01187435  0.02867767 -0.01247557  0.00430455
  0.0399867  -0.06506442 -0.08116747 -0.08637647  0.02510711  0.05003057
 -0.03711109 -0.1138487  -0.08131045 -0.03463127 -0.03673348  0.04569084
 -0.11320578 -0.08504802  0.02218035  0.11647724 -0.01486611  0.08850728
  0.09021524  0.04143781  0.02294724 -0.07972384  0.03615396 -0.01052252
  0.10134865  0.12215883 -0.11308592 -0.10592898  0.04805316  0.0019978
  0.07119112 -0.07260555 -0.00275785  0.04225174  0.02386512  0.02371062
  0.05208715  0.01778149  0.04157006  0.07553307  0.07370988 -0.01534177
 -0.03946494 -0.07568305 -0.02952859  0.05277713 -0.04279554 -0.01600035
  0.00296011 -0.05719727 -0.02668444 -0.06297645 -0.06833315  0.04777376
  0.01661924  0.04157096  0.02520042  0.00891429 -0.

In [164]:
list_of_sentences_test, list_of_sentences_with_tags_test, list_of_words_test = preprocess("dev.tagged", True)

In [165]:
model_dev = Word2Vec(sentences=list_of_sentences_test, vector_size=300, window=2, min_count=1, workers=4, epochs=100, seed=42)
model_dev.save("word2vec_dev.model")
model_dev.init_sims(replace=True)

C:\Users\shirel\AppData\Local\Temp\ipykernel_169868\201559699.py:3: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_dev.init_sims(replace=True)


In [166]:
representation_test = []
labels_test = []
for word, tag in list_of_words_test:
    if word not in word2vec_google.key_to_index:
        print(f"{word} // {tag} not an existing word in the model")
        vec = model_dev.wv[word]
    else:
        vec = word2vec_google[word]
        google +=1
    labels_test.append(tag)
    representation_test.append(vec)

? // 0 not an existing word in the model
´ // 0 not an existing word in the model
and // 0 not an existing word in the model
. // 0 not an existing word in the model
' // 0 not an existing word in the model
redondo // 1 not an existing word in the model
. // 0 not an existing word in the model
of // 0 not an existing word in the model
gardena // 1 not an existing word in the model
/ // 0 not an existing word in the model
compton // 1 not an existing word in the model
' // 0 not an existing word in the model
and // 1 not an existing word in the model
morty // 1 not an existing word in the model
😂 // 0 not an existing word in the model
and // 0 not an existing word in the model
kaite // 1 not an existing word in the model
and // 0 not an existing word in the model
😀 // 0 not an existing word in the model
😀 // 0 not an existing word in the model
😀 // 0 not an existing word in the model
😗 // 0 not an existing word in the model
😘 // 0 not an existing word in the model
😭 // 0 not an existing

In [167]:
from sklearn.neighbors import KNeighborsClassifier

KNN_classifier = KNeighborsClassifier(n_neighbors=5)
KNN = KNN_classifier.fit(representation, labels)
prediction = KNN.predict(representation_test)


In [168]:
from sklearn import metrics
print(metrics.classification_report(labels_test, prediction))

F1_Score=metrics.f1_score(labels_test, prediction)
print(f"F1 score = {F1_Score}")

              precision    recall  f1-score   support

           0       0.94      0.99      0.96     14483
           1       0.71      0.26      0.38      1250

    accuracy                           0.93     15733
   macro avg       0.83      0.63      0.67     15733
weighted avg       0.92      0.93      0.92     15733

F1 score = 0.3800586510263929


In [138]:
# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
# Running 4-Fold Cross validation on a given algorithm

Accuracy_Values=cross_val_score(KNN, representation_test , labels_test, cv=4, scoring = "f1")
print('\nF1 values for 4-fold Cross Validation:\n',Accuracy_Values)
print('\nFinal Average F1 of the model:', round(Accuracy_Values.mean(),2))


Accuracy values for 10-fold Cross Validation:
 [0.49828179 0.54026846 0.57142857 0.53160454]

Final Average Accuracy of the model: 0.54
